<a href="https://colab.research.google.com/github/ilhamije/multisensorHAR/blob/main/clientside_solution_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
print(tf.__version__)

2.6.0


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


`# Make sure the notebook has tensorflow-federated (Do it once)`
`!pip install --upgrade tensorflow-federated # comment out after installation `

In [4]:
def load_dataset(subject_id=3):
    df = pd.read_csv('drive/MyDrive/Colab Notebooks/assessment-hankuk/train_{}.csv'\
                     .format(subject_id))

    trainy = df['label']
    df_X = df.drop(['subject', 'label'], axis=1)

    loaded = list()
    loaded.append(df_X.values)
    trainX = np.dstack(loaded)
    trainy = to_categorical(trainy)

    return trainX, trainy

In [5]:
# Fit and evaluate a model
def evaluate_model(trainX, trainy):
    # set epochs and batch_size to 1 each due to its purpose solely as example and limiting resource
    # set verbose to 1 to see training progress
    verbose, epochs, batch_size = 1, 1, 1

    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    
    # Set precision and recall to calculate F1 score
    precision = tf.keras.metrics.Precision(name='precision')
    recall = tf.keras.metrics.Recall(name='recall')
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall])
    
    # fit network
    history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

    # get evaluation metrics
    accuracy = history.history['accuracy'][epochs-1]
    precision = history.history['precision'][epochs-1]
    recall = history.history['recall'][epochs-1]
    
    return accuracy, precision, recall, history

In [ ]:
# def plot_learningCurve(history, epochs):
#     # Plot training & validation accuracy values
#     epoch_range = range(1, epochs+1)
#     plt.plot(epoch_range, history.history['accuracy'])
#     plt.plot(epoch_range, history.history['val_accuracy'])
#     plt.title('Model accuracy')
#     plt.ylabel('Accuracy')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Val'], loc='upper left')
#     plt.show()

#     # Plot training & validation loss values
#     plt.plot(epoch_range, history.history['loss'])
#     plt.plot(epoch_range, history.history['val_loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Val'], loc='upper left')
#     plt.show()

In [6]:
# summarize scores
def summarize_results(scores, f1):
    m, s = np.mean(scores), np.std(scores)
    m2, s2 = np.mean(f1), np.std(f1)
    print('Accuracy: %.3f%% (+/-%.3f), F1 score: %.3f%% (+/-%.3f)' % (m, s, m2, s2))


In [7]:
# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy = load_dataset()

    # repeat experiment
    accuracies = list()
    f1s = list()
    for r in range(repeats):
        accuracy, precision, recall, history = evaluate_model(trainX, trainy)
        accuracy = accuracy * 100.0
        f1_score = (2.0*((precision * recall)/(precision + recall))) * 100.0
        print('> Iteration #{:3d}: {:.3f}, F1: {:.3f}'.format(r+1, accuracy, f1_score))
        accuracies.append(accuracy)
        f1s.append(f1_score)
        
    # summarize results
    summarize_results(accuracies, f1s)

In [9]:
df = pd.read_csv('drive/MyDrive/Colab-Notebooks/assessment-hankuk/train.csv')
subject_number = list(df['subject'].unique())

In [ ]:
run_experiment()

41984/41984 [==============================] - 87s 2ms/step - loss: 0.4950 - accuracy: 0.7944 - precision: 0.8586 - recall: 0.7273
> Iteration #  1: 79.437, F1: 78.755
Accuracy: 79.437% (+/-0.000), F1 score: 78.755% (+/-0.000)
